In [1]:
from google.cloud import storage  # 記得pip install google-cloud-storage==2.16.0
import pandas as pd
import os

In [2]:
# 設定環境變數 json檔案(後面路徑自己修改key放置的路徑)
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\TIR101_Group2\secrets\andy_GCS_viewer.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'D:\data_engineer\TIR_group2\TIR101_Group2\secrets\andy_GCS_viewer.json'

In [3]:
storage_client = storage.Client()             ###這是將所有我的bucket列出來  MRT的資料有以下幾張表(除了test以外的)
[b for b in storage_client.list_buckets()]     ##我一張大表建立一個bucket  bucket下面有很多csv檔案

[<Bucket: mrt_parking_info>,
 <Bucket: mrt_realtime_arrival>,
 <Bucket: mrt_realtime_crowded>,
 <Bucket: mrt_realtime_parking>,
 <Bucket: testbucket0204>]

In [4]:
# bucket = storage_client.bucket("youbike_static")#youbike資料(用harry的key)
bucket = storage_client.bucket("mrt_realtime_crowded")   #選擇你要用bucket的名字  (要在上個cell有列出的才行)

In [5]:
# 每格5分鐘打的資料會儲存成一張csv  檔名包含日期
# 利用bucket.list_blobs列出所有小表的檔名
blobs_list = [b.name for b in bucket.list_blobs()]  
blobs_list[:6]

['mrt_realtime_crowded20240508_1724.csv',
 'mrt_realtime_crowded20240508_1725.csv',
 'mrt_realtime_crowded20240508_1730.csv',
 'mrt_realtime_crowdedBL20240508_1910.csv',
 'mrt_realtime_crowdedBL20240508_1915.csv',
 'mrt_realtime_crowdedBL20240508_1920.csv']

In [7]:
for blob_name in blobs_list:  #用迴圈將每個csv表下載  #會下載一陣子
    blob = bucket.blob(blob_name)
    filename = os.path.join("C:\TIR101_Group2\DA\data\MRT",blob_name)   #下載到電腦裡放置資料的路徑(要自己改)
    blob.download_to_filename(filename)

In [10]:
filesnames = os.listdir(("C:\TIR101_Group2\DA\data\MRT"))  #列出剛剛下載所有csv路徑下的所有csv檔案名稱
df_list = [pd.read_csv(os.path.join("C:\TIR101_Group2\DA\data\MRT",f)) for f in filesnames] # 將每個檔案都讀進來放入list

In [11]:
df = pd.concat(df_list,axis=0) #將剛剛list裡面所有csv檔案接起來

,mrt_station_id,mrt_station_name,line_type,direction,cart1,cart2,cart3,cart4,cart5,cart6,update_time
0,G07,NaN,G,下行,1,2,2,2,2,1,2024-05-08 17:23:56
1,G10,NaN,G,下行,2,2,3,2,2,2,2024-05-08 17:23:59
2,G16,NaN,G,下行,1,1,1,1,2,2,2024-05-08 17:23:57
3,G18,NaN,G,上行,1,1,1,1,1,1,2024-05-08 17:20:49
4,G16,NaN,G,上行,1,2,1,1,1,1,2024-05-08 17:23:06
...,...,...,...,...,...,...,...,...,...,...,...
70,R13,NaN,R,上行,2,2,2,2,2,2,2024-05-10 09:34:02
71,R07,NaN,R,上行,1,1,1,1,1,1,2024-05-10 09:30:38
72,R20,NaN,R,上行,1,1,1,1,1,1,2024-05-10 09:34:19
73,R09,NaN,R,上行,2,2,1,2,2,1,2024-05-10 09:33:35
